In [35]:
import cv2
import numpy as np
import random


In [36]:
img = cv2.imread('Images/girl5.jpg')
imgInfo = img.shape
height = imgInfo[0]
width = imgInfo[1]
#cv2.imshow('empire',img)
# canny边缘检测 1.灰度处理 2.高斯滤波 3.canny算子
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# imgG = cv2.GaussianBlur(gray,(3,3),0)
# dst = cv2.Canny(img,50,50)          # (data,th1,th2)双阈值法 大于th2为边界点 小于th1为非边界点 图片经过卷积运算之后若>50，则属于边缘点
# cv2.imshow('dst',dst)
# cv2.waitKey(0)

In [37]:
# sobel算子
def Sobel():
    x = cv2.Sobel(gray,cv2.CV_16S,1,0)
    y = cv2.Sobel(gray,cv2.CV_16S,0,1)   

    absX = cv2.convertScaleAbs(x)
    absY = cv2.convertScaleAbs(y)

    dst = cv2.addWeighted(absX,0.5,absY,0.5,0)

    # cv2.imshow("absX", absX)
    # cv2.imshow("absY", absY)
    # cv2.imshow("dst", dst)
    # cv2.waitKey(0)
    return dst
Sobel()


13

In [38]:
# 基于sobel算子的Canny算法
import math
height = imgInfo[0]
width = imgInfo[1]
dst = np.zeros((height,width,1),np.uint8)
grd = np.zeros((height,width,2),np.uint8)
for i in range(0,height-2):                         # 以i j为左上角起点的3X3卷积核   sobel算子的由来平均梯度矢量 里面的距离是城市距离（D4）
    for j in range(0,width-2):
        gy = gray[i,j] + 2*gray[i,j+1] + gray[i,j+2] - gray[i+2,j] - 2*gray[i+2,j+1] - gray[i+2,j+2]
        gx = gray[i,j] + 2*gray[i+1,j] + gray[i+2,j] - gray[i,j+2] - 2*gray[i+1,j+2] - gray[i+2,j+2]
        #print(gx,gy)    
        grd[i,j] = (abs(gx),abs(gy))                #遇到负梯度，取绝对值
        grad = math.sqrt(gx**2 + gy**2)             # 计算梯度
        if grad > 50:
            dst[i,j] = 255
        else:
            dst[i,j] = 0
# 改进：引入45度方向和135度方向的梯度矢量，
cv2.imshow('dst',dst)           
cv2.waitKey(0)


13

In [43]:
# 图像叠加
def reduce_Cr(img):
    height = imgInfo[0]
    width = imgInfo[1]
    ycrcb = cv2.cvtColor(img,cv2.COLOR_BGR2YCrCb)
    Y = ycrcb[:,:,0]
    Cr = ycrcb[:,:,1]
    Cb = ycrcb[:,:,2]
    for i in range (0,height):
        for j in range(0,width):
            if(dst[i,j] == 255):
                Cr[i,j] = Cr[i,j]*0.9
    ycrcb[:,:,1] = Cr
    Result = cv2.cvtColor(ycrcb,cv2.COLOR_YCrCb2BGR)
    cv2.imshow('Result',Result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return Result

In [40]:
# 基于CRONE算子的边缘分割算法